# Лабораторная работа №5
## Выполнил студент группы БПИ2301 Спесивцев Даниил Владимирович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [2]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [8]:
def prefix_function(pattern: str):
    p = [0] * len(pattern)
    j = 0
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[j]:
            j += 1
            p[i] = j
            i += 1
        else:
            if j != 0:
                j = p[j-1]
            else:
                p[i] = 0
                i += 1
    return p

def kmp_find(text, pattern) -> int:    
    if not pattern or not text:
        return -1
    
    prefix_arr = prefix_function(pattern)
    i = 0
    j = 0
    while i < len(text):
        if pattern[j] == text[i]:
            i+=1
            j+=1
        if j == len(pattern):
            return i - j
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = prefix_arr[j-1]
            else:
                i+=1
    return -1


#### Упрощенный алгоритм Бойера-Мура

In [7]:
def offset(pattern: str) -> defaultdict:
    unique = set()
    length = len(pattern)
    off_dict = defaultdict()

    for i in range(length-2, -1, -1):
        if pattern[i] not in unique:
            off_dict[pattern[i]] = length-1-i
            unique.add(pattern[i])

    if pattern[-1] not in unique:
        off_dict[pattern[-1]] = length

    off_dict["*"] = length

    return off_dict

def bm_find(text, pattern) -> int:
    if not pattern or not text:
        return -1
    
    offset_dict = offset(pattern)
    text_length= len(text)
    pattern_length = len(pattern)
    if text_length <= pattern_length:
        return -1
    
    i = pattern_length - 1
    while i < text_length:
        k = 0
        flag = False
        for j in range(pattern_length-1, -1, -1):
            if pattern[j] != text[i-k]:
                if j == pattern_length-1:
                    off = offset_dict[text[i]] if offset_dict.get(text[i]) != None else offset_dict["*"]
                else:
                    off = offset_dict[pattern[j]]
            
                i += off
                flag = True
                break
            k += 1
        if not flag:
            return i - k + 1
                

 


In [9]:
from ipywidgets import interact, Text, Checkbox

def time(func, text, pattern):
    start = t.now()
    result = func(text, pattern)
    end = t.now()
    return result, (end - start).total_seconds()

def show_result(text, pattern, case_sensitive):
    if case_sensitive == False:
        text = text.lower()
        pattern = pattern.lower()

    standard_result, standard_time = time(lambda t, p: t.find(p), text, pattern)
    print(f"Стандартная функция поиска: индекс {standard_result}, время {standard_time:.6f} сек")

    kmp_result, kmp_time = time(kmp_find, text, pattern)
    print(f"Алгоритм Кнута-Морриса-Пратта: индекс {kmp_result}, время {kmp_time:.6f} сек")
        
    bm_result, bm_time = time(bm_find, text, pattern)
    print(f"Упрощенный алгоритм Бойера-Мура: индекс {bm_result}, время {bm_time:.6f} сек")

interact(show_result,
    text = Text(description="Введите строку: "),
    pattern = Text(description="Введите подстроку: "),
    case_sensitive = Checkbox(description="Чувствительность к регистру?")
)



interactive(children=(Text(value='', description='Введите строку: '), Text(value='', description='Введите подс…

<function __main__.show_result(text, pattern, case_sensitive)>

### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [6]:
from collections import deque

def get_neighbors(pos):
    x, y = pos
    moves = [(0,1), (0,-1), (1,0), (-1, 0)]
    return [(x + dx, y + dy) for dx, dy in moves if 0 <= x + dx < 4 and 0 <= y + dy < 4]

def is_solvable(puzzle):
    puzzle = [i for i in puzzle if i!=0]
    inversions = 0
    for i in range(len(puzzle)):
        for j in range(i+1, len(puzzle)):
            if puzzle[i]>puzzle[j]:
                inversions += 1
    return inversions % 2 == 0

def solve(puzzle):
    if is_solvable(puzzle) == False:
        return []
    
    final_state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]

    if puzzle == final_state:
        return []
    
    puzzle_copy = puzzle[:]
    queue = deque([(puzzle_copy, [])])
    visited = set()

    while queue:
        state, path = queue.popleft()
        if state == final_state:
            return path

        zero_pos = state.index(0)
        zero_x, zero_y = zero_pos // 4, zero_pos % 4

        for new_x, new_y in get_neighbors((zero_x, zero_y)):
            new_zero_pos = new_x * 4 + new_y
            new_state = state[:]
            new_state[zero_pos], new_state[new_zero_pos] = new_state[new_zero_pos], new_state[zero_pos]

            if str(new_state) not in visited:
                visited.add(str(new_state))
                queue.append((new_state, path + [state[new_zero_pos]]))

    return []

puzzle = [
    1, 2, 3, 4,
    5, 6, 7, 8,
    9, 11, 10, 12,
    13, 15, 14, 0
]

solve(puzzle)


[14, 10, 11, 15, 10, 14, 12, 11, 15, 10, 14, 15, 11, 12]